In [1]:
import os
import tensorflow as tf
import numpy as np
import tvm
import tvm.relay as relay
from riptide.get_models import get_model
from riptide.binary.binary_layers import Config, DQuantize, XQuantize

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [3]:
config = Config(actQ=DQuantize, weightQ=XQuantize, bits=1, use_act=False, use_bn=False, use_maxpool=True)

In [4]:
with config:
    model = get_model('vggnet')

In [5]:
test_input = tf.keras.Input(shape=[32, 32, 3], dtype='float32')
output = model(test_input)

W0603 03:49:53.878260 140550309254976 deprecation.py:323] From /home/Riptide/riptide/binary/binary_layers.py:526: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
func, params = relay.frontend.from_keras(model)

v0.0.1
free_var %input_1
%0 = nn.pad(%input_1, pad_width=[[0, 0], [0, 0], [2, 3], [2, 3]])
free_var %v_param_1: Tensor[(96, 3, 7, 7), float32]
%1 = nn.conv2d(%0, %v_param_1, strides=[2, 2], channels=96, kernel_size=[7, 7])
%2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2])
free_var %v_param_2: Tensor[(96,), float32]
free_var %v_param_3: Tensor[(96,), float32]
%3 = nn.batch_norm(%2, 1f, 0f, %v_param_2, %v_param_3, epsilon=0.001, center=False, scale=False)
%4 = %3.0
%5 = multiply(%4, 1f)
%6 = clip(%5, a_min=0, a_max=1)
%7 = multiply(1f, %6)
round(%7)


OpNotImplemented: Operator BinaryConv2D is not supported for frontend Keras.

In [9]:
model.layers[0].inbound_nodes[0].node_indices

0

In [7]:
model.summary()

Model: "vggnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 96)        14112     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 96)          0         
_________________________________________________________________
batch_normalization_v2 (Batc (None, 8, 8, 96)          192       
_________________________________________________________________
enter_integer (EnterInteger) (None, 8, 8, 96)          0         
_________________________________________________________________
binary_conv2d (BinaryConv2D) (None, 8, 8, 256)         221184    
_________________________________________________________________
shift_normalization (ShiftNo (None, 8, 8, 256)         512       
_________________________________________________________________
binary_conv2d_1 (BinaryConv2 (None, 8, 8, 256)         58982